# Exploratory parsing

Data source: 
- Änderungsgesetz: https://dip.bundestag.de/experten-suche?term=NOT%20zusatzmerkmal:E%20AND%20vorgangstyp_notation:100&f.wahlperiode=19&f.typ=Vorgang&rows=25&sort=datum_ab
- Source law: https://www.gesetze-im-internet.de/wregg/BJNR273910017.html

Use anytree python package for tree datastructure: https://anytree.readthedocs.io/en/latest/

## Read data

In [1]:
import pdfplumber
import regex as re

In [2]:
# open pdf
filename = '../data/0483-21.pdf'

def read_pdf_law(filename: str) -> str:
    # read all pages
    pdf_file_obj = pdfplumber.open(filename)

    # join the pages and remove newlines
    return "".join([page for page in [page.extract_text() for page in pdf_file_obj.pages] if page])


change_law_raw = read_pdf_law(filename)

## Structure and clean up the text

In [118]:
def preprocess_raw_law(text: str) -> str:
    # extract the change requests: get the text between "wird wie folgt geändert" und "Begründung"
    text = re.split(r"wird[\s,\n]{0,3}wie[\s,\n]{0,3}folgt[\s,\n]{0,3}geändert:", text, maxsplit=1)[1].split("Begründung", 1)[0]

    # remove newlines between quotation marks
    for m in re.finditer(r"(?<=„)(.|\n)*?(?=“)", text, re.MULTILINE):
        text = text[:m.span()[0]] + text[m.span()[0]:m.span()[1]].replace("\n", " ") + text[m.span()[1]:]

    # remove other text from the page a bit
    text = re.sub(r"\.?Drucksache \d{2,3}\/\d{1,2}", "", text)
    text = re.sub(r"- \d -", "", text)
    text = text.strip()
    
    outtext = ""
    
    # pull every bulletpoint content to one line
    for line_num, line in enumerate(text.split("\n")):
        if any([
            re.match(r"^\d\.", line),
            re.match(r"^[a-z]\)", line),
            re.match(r"^[a-z][a-z]\)", line),
            re.match(r"^\([a-z1-9]\)", line)
        ]):
            outtext += "\n" + line
        else:
            outtext += line
            
    return outtext.strip()


clean_change_law = preprocess_raw_law(change_law_raw)

In [161]:
# expand text for multi-operation bullet points
def expand_text(text: str) -> str:
    outtext = ""
    stack = []
    for line in text.split("\n"):
        if re.match(r"^\d\.", line):
            if stack:
                outtext += " ".join(stack) + "\n"
            stack = []
            stack.append(line)
        if re.match(r"^[a-z]\)", line):
            if len(stack) >= 2:
                outtext += " ".join(stack) + "\n"
                stack.pop()
            stack.append(line)
        if re.match(r"^[a-z][a-z]\)", line):
            if len(stack) >= 3:
                outtext += " ".join(stack) + "\n"
                stack.pop()
            stack.append(line)
    if stack:
        outtext += " ".join(stack) + "\n"
    return outtext


clean_text = expand_text(clean_change_law)

In [162]:
print(clean_text)

1.  § 2 wird wie folgt geändert:  a)  Nach Absatz 2 wird folgender neuer Absatz 3 eingefügt: „(3) In  das  Wettbewerbsregister  werden  ferner  Verfehlungen  von  vergleichbarer  Schwere  wie  die  Straf-  und  Ordnungswidrigkeitstatbestände  der  Absätze  1  und  2  (schwere  Verfehlungen)  eingetragen,  insbesondere  vorsätzliche  oder  grob  fahrlässige Falscherklärungen  1.  zum  Vorliegen  von  Registereintragungen  nach  § 2  Absatz 1  und 2 oder in vergleichbaren Registern,  2.  zur Einhaltung der Tariftreue und der Bestimmungen über einen  gesetzlichen Mindestlohn oder  3.  zur Berücksichtigung der Kernarbeitsnormen der Internationalen  Arbeitsorganisation.  Der für die Eintragung erforderliche Nachweis der jeweiligen schweren  Verfehlung gilt als erbracht, wenn angesichts der Tatsachenlage kein  vernünftiger  Zweifel  am  Vorliegen  einer  schweren  Verfehlung  verbleibt.  Die  Feststellung  hat  die  nach  Landesrecht  zuständige  Behörde zu treffen.“
1.  § 2 wird wie folgt g

## Parse changes

In [239]:
from typing import List


def parse_change_location(line: str) -> List[str]:
    """Parse the location identifiers from one line of change request."""
    location = []
    location_identifiers = [
        r"Inhaltsübersicht?",
        r"§ \d{1,3}[a-z]?",
        r"Überschrift",
        r"Absatz \d{1,3}",
        r"Satz \d{1,3}",
        r"Nummer \d{1,3}",
        r"Buchstabe [a-z]"
    ]
    for loc_ident in location_identifiers:
        try:
            location.extend(re.search(loc_ident, re.sub(r"(?<=„)(.|\n)*?(?=“)", "", line)).captures())
        except:
            pass
    return location


def parse_change_text(line: str) -> List[str]:
    """Parse the text that needs to be changed from one line of change request."""
    return [line[m.span()[0]:m.span()[1]]
            for m in re.finditer(r"(?<=„)(.|\n)*?(?=“)", line, re.MULTILINE)]


def parse_change_request_line(line: str) -> dict:
    """Parse the actions of one line of change requests."""
    res = []
    if "eingefügt" in line:
        res_dict = {
            "location": parse_change_location(line),
            "text": parse_change_text(line),
            "how": "insert_after"
        }
        res.append(res_dict)
    elif "ersetzt" in line:
        res_dict = {
            "location": parse_change_location(line),
            "text": parse_change_text(line),
            "how": "replace"
        }
        res.append(res_dict)
    elif "gefasst" in line:
        res_dict = {
            "location": parse_change_location(line),
            "text": parse_change_text(line),
            "how": "rephrase"
        }
        res.append(res_dict)
    elif "angefügt" in line:
        res_dict = {
            "location": parse_change_location(line),
            "text": parse_change_text(line),
            "how": "append"
        }
        res.append(res_dict)
    elif "gestrichen" in line:
        res_dict = {
            "location": parse_change_location(line),
            "text": parse_change_text(line),
            "how": "delete_after"
        }
        res.append(res_dict)
    elif "aufgehoben" in line:
        res_dict = {
            "location": parse_change_location(line),
            "text": parse_change_text(line),
            "how": "cancelled"
        }
        res.append(res_dict)
    else:
        res_dict = {
            "location": parse_change_location(line),
            "text": parse_change_text(line),
            "how": "UNKNOWN"
        }
        res.append(res_dict)

    return res

In [240]:
changes = []
for change_request_line in clean_text.split("\n"):
    res = parse_change_request_line(change_request_line)
    print(res)
    changes.extend(res)
    print()

[{'location': ['§ 2', 'Absatz 2'], 'text': ['(3) In  das  Wettbewerbsregister  werden  ferner  Verfehlungen  von  vergleichbarer  Schwere  wie  die  Straf-  und  Ordnungswidrigkeitstatbestände  der  Absätze  1  und  2  (schwere  Verfehlungen)  eingetragen,  insbesondere  vorsätzliche  oder  grob  fahrlässige Falscherklärungen  1.  zum  Vorliegen  von  Registereintragungen  nach  § 2  Absatz 1  und 2 oder in vergleichbaren Registern,  2.  zur Einhaltung der Tariftreue und der Bestimmungen über einen  gesetzlichen Mindestlohn oder  3.  zur Berücksichtigung der Kernarbeitsnormen der Internationalen  Arbeitsorganisation.  Der für die Eintragung erforderliche Nachweis der jeweiligen schweren  Verfehlung gilt als erbracht, wenn angesichts der Tatsachenlage kein  vernünftiger  Zweifel  am  Vorliegen  einer  schweren  Verfehlung  verbleibt.  Die  Feststellung  hat  die  nach  Landesrecht  zuständige  Behörde zu treffen.'], 'how': 'insert_after'}]

[{'location': ['§ 2'], 'text': [], 'how': 'UNK

## Parse the source law from raw text

In [194]:
source_text = open("../data/0483-21_wettbewerbsgesetzt.txt","r").read()

In [224]:
def parse_source_law(text: str) -> dict:
    res = dict()
    patterns = [r"\n§\s\d{1,2}", r"\n\s*\([a-z1-9]\)", r"\n\s*\d{1,2}\.", r"\n\s*[a-z]\)"]
    for pattern in patterns:
        if re.search(pattern, text):
            split_text = re.split(pattern, text)
            for idx, m in enumerate(re.finditer(pattern, text)):
                res[text[m.span()[0]: m.span()[1]].strip()] = {"text": split_text[idx + 1].strip().split("\n")[0], "content": parse_source_law(split_text[idx + 1])}
    return res

    
parsed_law = parse_source_law(source_text)
parsed_law["§ 2"]

{'text': 'Eintragungsvoraussetzungen',
 'content': {'(1)': {'text': 'In das Wettbewerbsregister sind einzutragen:',
   'content': {'1.': {'text': 'rechtskräftige strafgerichtliche Verurteilungen und Strafbefehle, die wegen einer der folgenden Straftaten ergangen sind:',
     'content': {'a)': {'text': 'in § 123 Absatz 1 des Gesetzes gegen Wettbewerbsbeschränkungen aufgeführte Straftaten,',
       'content': {}},
      'b)': {'text': 'Betrug nach § 263 des Strafgesetzbuchs und Subventionsbetrug nach § 264 des Strafgesetzbuchs, soweit sich die Straftat gegen öffentliche Haushalte richtet,',
       'content': {}},
      'c)': {'text': 'Vorenthalten und Veruntreuen von Arbeitsentgelt nach § 266a des Strafgesetzbuchs,',
       'content': {}},
      'd)': {'text': 'Steuerhinterziehung nach § 370 der Abgabenordnung oder',
       'content': {}},
      'e)': {'text': 'wettbewerbsbeschränkende Absprachen bei Ausschreibungen nach § 298 des Strafgesetzbuchs;',
       'content': {}}}},
    '2.': {'

## Apply the changes to the source law

In [241]:
# apply first change request
changes[0]

{'location': ['§ 2', 'Absatz 2'],
 'text': ['(3) In  das  Wettbewerbsregister  werden  ferner  Verfehlungen  von  vergleichbarer  Schwere  wie  die  Straf-  und  Ordnungswidrigkeitstatbestände  der  Absätze  1  und  2  (schwere  Verfehlungen)  eingetragen,  insbesondere  vorsätzliche  oder  grob  fahrlässige Falscherklärungen  1.  zum  Vorliegen  von  Registereintragungen  nach  § 2  Absatz 1  und 2 oder in vergleichbaren Registern,  2.  zur Einhaltung der Tariftreue und der Bestimmungen über einen  gesetzlichen Mindestlohn oder  3.  zur Berücksichtigung der Kernarbeitsnormen der Internationalen  Arbeitsorganisation.  Der für die Eintragung erforderliche Nachweis der jeweiligen schweren  Verfehlung gilt als erbracht, wenn angesichts der Tatsachenlage kein  vernünftiger  Zweifel  am  Vorliegen  einer  schweren  Verfehlung  verbleibt.  Die  Feststellung  hat  die  nach  Landesrecht  zuständige  Behörde zu treffen.'],
 'how': 'insert_after'}

In [243]:
def get_node(change_spec, parsed_law):
    current_node = parsed_law.copy()
    for loc in change_spec["location"]:
        if loc.startswith("Absatz "):
            loc = loc.replace("Absatz ", "(") + ")"
        elif loc.startswith("Nummer "):
            loc = loc.replace("Nummer ", "") + "."
        elif loc.startswith("Buchstabe "):
            loc = loc.replace("Buchstabe ", "") + ")"
        if len(current_node[loc]["content"]) == 0:
            current_node = current_node[loc]
            break
        current_node = current_node[loc]["content"]
    return current_node

i = 4
print(i)
print(changes[i])
print(get_node(changes[i], parsed_law))

4
{'location': ['§ 3', 'Absatz 1', 'Nummer 5', 'Buchstabe d'], 'text': ['§ 2 Absatz 3 Satz 2', '§ 2 Absatz 4 Satz 2'], 'how': 'replace'}
{'text': 'die die Zurechnung des Fehlverhaltens zu einem Unternehmen gemäß § 2 Absatz 3 Satz 2 begründenden Umstände sowie', 'content': {}}


In [233]:
def apply_change(change_spec, parsed_law) -> str:
    
    # resolve insert after
    if change_spec["how"] == "replace":
        for loc in change_spec["location"]:
            if loc.startswith("Absatz "):
                loc = loc.replace("Absatz ", "(") + ")"
            elif loc.startswith("Nummer "):
                loc = loc.replace("Nummer ", "") + "."
            elif loc.startswith("Buchstabe "):
                loc = loc.replace("Buchstabe ", "") + ")"
    return "\n".join(res_text)

In [244]:
res = apply_change(changes[4], parsed_law)

print(res)

NameError: name 'res_text' is not defined

## Apply it all to another document

In [195]:
# try a second document
filename2 = '../data/0145-21.pdf'

change_law_raw2 = read_pdf_law(filename2)

In [201]:
clean_change_law2 = preprocess_raw_law(change_law_raw2)
clean_text2 = expand_text(clean_change_law2)

for t in clean_text2.split("\n")[:15]:
    print(t)

1. Die Inhaltsübersicht wird wie folgt geändert: a) Der Angabe zu § 130a werden ein Semikolon und das Wort „Verordnungsermäch- tigung“ angefügt.
1. Die Inhaltsübersicht wird wie folgt geändert: b) Die Angaben zu den §§ 173 bis 176 werden wie folgt gefasst:„§ 173 Zustellung elektronischer Dokumente §174 Zustellung durch Aushändigung an der Amtsstelle §175 Zustellung eines Schriftstücks gegen Empfangsbekenntnis §176 Zustellung durch Einschreiben mit Rückschein; Zustellungsauftrag“.
2. § 130a wird wie folgt geändert: a) Der Überschrift werden ein Semikolon und das Wort „Verordnungsermächtigung“angefügt.
2. § 130a wird wie folgt geändert: b) Absatz 2 Satz 2 wird wie folgt gefasst:„Die Bundesregierung bestimmt durch Rechtsverordnung mit Zustimmung des Bundesrates technische Rahmenbedingungen für die Übermittlung und Eignung zur Bearbeitung durch das Gericht.“
2. § 130a wird wie folgt geändert: c) Absatz 4 wird wie folgt geändert: aa) In Nummer 3 werden nach dem Wort „Gerichts“ das Semikolon

In [202]:
changes = []
for change_request_line in clean_text2.split("\n"):
    res = parse_change_request_line(change_request_line)
    print(change_request_line)
    print()
    print(res)
    changes.extend(res)
    print()
    print(50*"#")
    print()

1. Die Inhaltsübersicht wird wie folgt geändert: a) Der Angabe zu § 130a werden ein Semikolon und das Wort „Verordnungsermäch- tigung“ angefügt.

[{'location': ['Inhaltsübersicht', '§ 130a'], 'text': ['Verordnungsermäch- tigung'], 'how': 'append'}]

##################################################

1. Die Inhaltsübersicht wird wie folgt geändert: b) Die Angaben zu den §§ 173 bis 176 werden wie folgt gefasst:„§ 173 Zustellung elektronischer Dokumente §174 Zustellung durch Aushändigung an der Amtsstelle §175 Zustellung eines Schriftstücks gegen Empfangsbekenntnis §176 Zustellung durch Einschreiben mit Rückschein; Zustellungsauftrag“.

[{'location': ['Inhaltsübersicht', '§ 173'], 'text': ['§ 173 Zustellung elektronischer Dokumente §174 Zustellung durch Aushändigung an der Amtsstelle §175 Zustellung eines Schriftstücks gegen Empfangsbekenntnis §176 Zustellung durch Einschreiben mit Rückschein; Zustellungsauftrag'], 'how': 'rephrase'}]

##################################################

